## Which Stats Contribute Most To Winning Matches

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import statsmodels.api as sm
import statsmodels.formula.api as smf
from scipy import stats
import matplotlib.pyplot as plt
from pandasql import sqldf
import sqlite3 
import datetime as dt
import plotly.express as px

In [2]:
con = sqlite3.connect('/Users/julianbombard/Desktop/Data Analytics Course/Capstone/database/ATP_Tour')
df = pd.read_sql_query('''Select *
                       From cleaned_match_stats''', con)

con.close()

df.head()

,player_id,match_id,year,month,player_name,country,height,hand,tourney_name,tourney_level,...,bp_saved,bp_faced,rank,rank_points,won,first_sv_pct,first_sv_win_pct,second_sv_win_pct,total_sv_win_pct,bp_save_pct
0,105357,2015-339_1,2015,1,John Millman,AUS,183,R,Brisbane,A,...,1,1,153,328,1,0.545455,0.791667,0.700000,0.750000,1.000000
1,103813,2015-339_2,2015,1,Jarkko Nieminen,FIN,185,L,Brisbane,A,...,4,7,73,689,1,0.641304,0.661017,0.515152,0.608696,0.571429
2,105902,2015-339_3,2015,1,James Duckworth,AUS,183,R,Brisbane,A,...,2,3,125,430,1,0.600000,0.740741,0.611111,0.688889,0.666667
3,104871,2015-339_4,2015,1,Jeremy Chardy,FRA,188,R,Brisbane,A,...,0,0,31,1195,1,0.735849,0.794872,0.785714,0.792453,NaN
4,105373,2015-339_5,2015,1,Martin Klizan,SVK,191,L,Brisbane,A,...,6,8,34,1094,1,0.607692,0.696203,0.529412,0.630769,0.750000


In [3]:
df['won'] = df['won'].astype('bool')

## Correlations

In [4]:
stat_columns = df.select_dtypes(include = [np.number, 'bool'])

correlations = stat_columns.corr()['won'].drop(['won', 'month', 'year']).sort_values(ascending = False)
print(correlations)

total_sv_win_pct     0.611462
first_sv_win_pct     0.509953
second_sv_win_pct    0.462707
bp_save_pct          0.299219
rank_points          0.198847
ace                  0.158791
first_won            0.135203
first_sv_pct         0.119937
second_won           0.110232
height               0.064227
sv_gms               0.027185
player_id            0.009595
minutes             -0.000975
first_in            -0.015436
svpt                -0.051525
df                  -0.133026
rank                -0.152069
bp_saved            -0.203976
bp_faced            -0.394425
Name: won, dtype: float64


## T- Tests

In [5]:
winners = df[df['won'] == True]
losers = df[df['won'] == False]

In [6]:
test_stats = stat_columns.columns.tolist()

In [7]:
for i in test_stats:
    winners = winners.dropna()
    losers = losers.dropna()
    stat, p_value = stats.ttest_ind(
        winners[i],
        losers[i],
        equal_var = False
    )
    print(f"{i:15} | p = {p_value:.4f}")
   



player_id       | p = 0.0228
year            | p = 0.9498
month           | p = 0.5493
height          | p = 0.0000
minutes         | p = 0.0000
ace             | p = 0.0000
df              | p = 0.0000
svpt            | p = 0.9312
first_in        | p = 0.0000
first_won       | p = 0.0000
second_won      | p = 0.0000
sv_gms          | p = 0.0000
bp_saved        | p = 0.0000
bp_faced        | p = 0.0000
rank            | p = 0.0000
rank_points     | p = 0.0000
won             | p = 0.0000
first_sv_pct    | p = 0.0000
first_sv_win_pct | p = 0.0000
second_sv_win_pct | p = 0.0000
total_sv_win_pct | p = 0.0000
bp_save_pct     | p = 0.0000


/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/scipy/stats/_axis_nan_policy.py:586: RuntimeWarning: Precision loss occurred in moment calculation due to catastrophic cancellation. This occurs when the data are nearly identical. Results may be unreliable.
  res = hypotest_fun_out(*samples, **kwds)


it seems that basically ever statistic has extremely sigfnicant contribution towards winning  